## DATA FETCH FROM ERA5

### Time Period
Data yang diambil akan diambil data 20 tahun untuk averaging faktor-faktor musiman seperti el-nino, dll. 

### Data yang dibutuhkan untuk solar

- Surface Solar Radiation Downwards (SSRD) 
Untuk menghitung berapa energi solar yang sampai ke permukaan bumi. 
- 2m Temperature (t2m)
Semakin tinggi temperature, semakin rendah efisiensi solar panel


### Data yang dibutuhkan untuk wind

- u-component of wind
- v-component of wind

Dipakai untuk menghitung vektor kecepatan angin. 

In [1]:
%pip install cdsapi
%pip install pandas xarray cfgrib numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import cdsapi
import pandas as pd
import os
import glob
import xarray as xr
import numpy as np

In [4]:
# Params

#Data Fetch
START_YEAR = 2019
END_YEAR = 2024
MONTH = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
# Pressure levels relevant for wind turbines (50-1000 hPa in increments of 25)
PRESSURE_LEVELS = [875, 900, 925, 950, 975, 1000]
DATASET = "reanalysis-era5-pressure-levels"
OUTPUT_FOLDER = "../data/wind_data_all_levels/"

#GRIB to df
INPUT_FOLDER = "../data/wind_data_all_levels/"

In [5]:
#Fetch data from cds

# Ensure the output directory exists
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Initialize the API client
client = cdsapi.Client()

# Generate a list of the first day of each month for the specified year range
date_range = pd.date_range(start=f'{START_YEAR}-01-01', end=f'{END_YEAR}-12-31', freq='MS')  # Monthly start

# Loop through each month
for date in date_range:
    year = date.strftime("%Y")
    month = date.strftime("%m")
    
    output_filename = f"wind_data_{year}_{month}_all_levels.grib"
    output_file_path = os.path.join(OUTPUT_FOLDER, output_filename)
    
    if os.path.exists(output_file_path):
        print(f"Skipping {output_filename}, file already exists.")
        continue

    print(f"--- Downloading data for {year}-{month} (all pressure levels) ---")

    try:
        request = {
            "product_type": "reanalysis",
            "format": "grib",
            "variable": [
                "u_component_of_wind",
                "v_component_of_wind"
            ],
            'pressure_level': PRESSURE_LEVELS,  # Use all pressure levels
            "year": year,
            "month": month,
            "day": [str(d).zfill(2) for d in range(1, 32)], # API handles non-existent days
            'time': [f'{h:02d}:00' for h in range(0, 24, 6)], # Every 6 hours to reduce data size
            "area": [6, 95, -11, 141], # North, West, South, East (Indonesia region)
        }

        # Retrieve and download the data
        client.retrieve(DATASET, request, output_file_path)
        
        print(f"Successfully downloaded to: {output_file_path}")

    except Exception as e:
        print(f"!!! FAILED to download data for {year}-{month}. Error: {e}")
      
print("\n--- Automation complete. ---")

2025-07-30 10:04:27,518 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


--- Downloading data for 2019-01 (all pressure levels) ---


2025-07-30 10:04:28,320 INFO Request ID is 01bfe206-6b1d-497a-9703-3f68f118136a
2025-07-30 10:04:28,684 INFO status has been updated to accepted
2025-07-30 10:04:28,684 INFO status has been updated to accepted
2025-07-30 10:04:38,825 INFO status has been updated to running
2025-07-30 10:04:38,825 INFO status has been updated to running
2025-07-30 10:07:25,288 INFO status has been updated to successful
2025-07-30 10:07:25,288 INFO status has been updated to successful


178e81773cdcf218b73e6378d5bd3a07.grib:   0%|          | 0.00/36.4M [00:00<?, ?B/s]

Successfully downloaded to: ../data/wind_data_all_levels/wind_data_2019_01_all_levels.grib
--- Downloading data for 2019-02 (all pressure levels) ---


...

30dc99567f2aa85ad2207585687a6938.grib:   0%|          | 0.00/35.2M [00:00<?, ?B/s]

Successfully downloaded to: ../data/wind_data_all_levels/wind_data_2024_11_all_levels.grib
--- Downloading data for 2024-12 (all pressure levels) ---


2025-07-30 14:56:29,349 INFO Request ID is 0f4beeac-10ef-4916-8139-cbf877a9fc7b
2025-07-30 14:56:29,686 INFO status has been updated to accepted
2025-07-30 14:56:29,686 INFO status has been updated to accepted
2025-07-30 14:56:35,565 INFO status has been updated to running
2025-07-30 14:56:35,565 INFO status has been updated to running
2025-07-30 14:58:26,873 INFO status has been updated to successful
2025-07-30 14:58:26,873 INFO status has been updated to successful


185422f69cf34d0b54d4bcdf6110d10c.grib:   0%|          | 0.00/36.4M [00:00<?, ?B/s]

Successfully downloaded to: ../data/wind_data_all_levels/wind_data_2024_12_all_levels.grib

--- Automation complete. ---


In [6]:
# Convert to df and coarse. 

#Note: Data increment in 0.25 Coarsen factor for now is 2 / increment per 0.5


# Find all .grib files
grib_files = sorted(glob.glob(os.path.join(INPUT_FOLDER, '*.grib')))

if not grib_files:
    print(f"Error: No .grib files found in '{INPUT_FOLDER}'")
else:
    print(f"Found {len(grib_files)} GRIB files to process.")
    
    all_coarse_dataframes = []
    for file_path in grib_files:
        print(f"Processing and coarsening {os.path.basename(file_path)}...")
        try:
            with xr.open_dataset(file_path, engine='cfgrib') as ds:
                # Coarsen the dataset (reduce spatial resolution)
                coarse_ds = ds.coarsen(latitude=2, longitude=2, boundary='trim').mean()
                
                # Convert to dataframe
                df = coarse_ds.to_dataframe()
                
                # Add filename info for tracking
                df['source_file'] = os.path.basename(file_path)
                
                all_coarse_dataframes.append(df)
                
        except Exception as e:
            print(f"!!! Could not process {file_path}. Error: {e}")

    if all_coarse_dataframes:
        combined_df = pd.concat(all_coarse_dataframes, ignore_index=False).reset_index()
        
        # Calculate wind speed from u and v components
        combined_df['wind_speed'] = np.sqrt(combined_df['u']**2 + combined_df['v']**2)
        
        # Calculate wind direction (in degrees from north)
        combined_df['wind_direction'] = np.arctan2(combined_df['u'], combined_df['v']) * 180 / np.pi
        combined_df['wind_direction'] = (combined_df['wind_direction'] + 360) % 360  # Convert to 0-360 degrees

        print("\n--- Processing Complete --- ✅")
        print("Final Coarse DataFrame Info:")
        print(f"Shape: {combined_df.shape}")
        print(f"Pressure levels: {sorted(combined_df['isobaricInhPa'].unique())}")
        print(f"Time range: {combined_df['time'].min()} to {combined_df['time'].max()}")
        print("\nDataFrame Head:")
        print(combined_df.head())

        

Found 72 GRIB files to process.
Processing and coarsening wind_data_2019_01_all_levels.grib...
Processing and coarsening wind_data_2019_02_all_levels.grib...
Processing and coarsening wind_data_2019_02_all_levels.grib...
Processing and coarsening wind_data_2019_03_all_levels.grib...
Processing and coarsening wind_data_2019_03_all_levels.grib...
Processing and coarsening wind_data_2019_04_all_levels.grib...
Processing and coarsening wind_data_2019_04_all_levels.grib...
Processing and coarsening wind_data_2019_05_all_levels.grib...
Processing and coarsening wind_data_2019_05_all_levels.grib...
Processing and coarsening wind_data_2019_06_all_levels.grib...
Processing and coarsening wind_data_2019_06_all_levels.grib...
Processing and coarsening wind_data_2019_07_all_levels.grib...
Processing and coarsening wind_data_2019_07_all_levels.grib...
Processing and coarsening wind_data_2019_08_all_levels.grib...
Processing and coarsening wind_data_2019_08_all_levels.grib...
Processing and coarseni

skipping corrupted Message
Traceback (most recent call last):
  File "d:\anaconda3\Lib\site-packages\cfgrib\messages.py", line 274, in itervalues
    yield self.filestream.message_from_file(file, errors=errors)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda3\Lib\site-packages\cfgrib\messages.py", line 341, in message_from_file
    return Message.from_file(file, offset, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda3\Lib\site-packages\cfgrib\messages.py", line 97, in from_file
    codes_id = eccodes.codes_grib_new_from_file(file)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda3\Lib\site-packages\gribapi\gribapi.py", line 415, in grib_new_from_file
    GRIB_CHECK(err)
  File "d:\anaconda3\Lib\site-packages\gribapi\gribapi.py", line 232, in GRIB_CHECK
    errors.raise_grib_error(errid)
  File "d:\anaconda3\Lib\site-packages\gribapi\errors.py", line 381, in raise_grib_error
    raise ERROR_M

Processing and coarsening wind_data_2021_10_all_levels.grib...
Processing and coarsening wind_data_2021_11_all_levels.grib...
Processing and coarsening wind_data_2021_11_all_levels.grib...
Processing and coarsening wind_data_2021_12_all_levels.grib...
Processing and coarsening wind_data_2021_12_all_levels.grib...


skipping corrupted Message
Traceback (most recent call last):
  File "d:\anaconda3\Lib\site-packages\cfgrib\messages.py", line 274, in itervalues
    yield self.filestream.message_from_file(file, errors=errors)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda3\Lib\site-packages\cfgrib\messages.py", line 341, in message_from_file
    return Message.from_file(file, offset, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda3\Lib\site-packages\cfgrib\messages.py", line 97, in from_file
    codes_id = eccodes.codes_grib_new_from_file(file)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda3\Lib\site-packages\gribapi\gribapi.py", line 415, in grib_new_from_file
    GRIB_CHECK(err)
  File "d:\anaconda3\Lib\site-packages\gribapi\gribapi.py", line 232, in GRIB_CHECK
    errors.raise_grib_error(errid)
  File "d:\anaconda3\Lib\site-packages\gribapi\errors.py", line 381, in raise_grib_error
    raise ERROR_M

Processing and coarsening wind_data_2022_01_all_levels.grib...


...

Processing and coarsening wind_data_2022_02_all_levels.grib...


skipping corrupted Message
Traceback (most recent call last):
  File "d:\anaconda3\Lib\site-packages\cfgrib\messages.py", line 274, in itervalues
    yield self.filestream.message_from_file(file, errors=errors)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda3\Lib\site-packages\cfgrib\messages.py", line 341, in message_from_file
    return Message.from_file(file, offset, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda3\Lib\site-packages\cfgrib\messages.py", line 97, in from_file
    codes_id = eccodes.codes_grib_new_from_file(file)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda3\Lib\site-packages\gribapi\gribapi.py", line 415, in grib_new_from_file
    GRIB_CHECK(err)
  File "d:\anaconda3\Lib\site-packages\gribapi\gribapi.py", line 232, in GRIB_CHECK
    errors.raise_grib_error(errid)
  File "d:\anaconda3\Lib\site-packages\gribapi\errors.py", line 381, in raise_grib_error
    raise ERROR_M

Processing and coarsening wind_data_2022_03_all_levels.grib...
Processing and coarsening wind_data_2022_04_all_levels.grib...
Processing and coarsening wind_data_2022_04_all_levels.grib...
Processing and coarsening wind_data_2022_05_all_levels.grib...
Processing and coarsening wind_data_2022_05_all_levels.grib...
Processing and coarsening wind_data_2022_06_all_levels.grib...
Processing and coarsening wind_data_2022_06_all_levels.grib...
Processing and coarsening wind_data_2022_07_all_levels.grib...
Processing and coarsening wind_data_2022_07_all_levels.grib...
Processing and coarsening wind_data_2022_08_all_levels.grib...
Processing and coarsening wind_data_2022_08_all_levels.grib...
Processing and coarsening wind_data_2022_09_all_levels.grib...
Processing and coarsening wind_data_2022_09_all_levels.grib...
Processing and coarsening wind_data_2022_10_all_levels.grib...
Processing and coarsening wind_data_2022_10_all_levels.grib...
Processing and coarsening wind_data_2022_11_all_levels.

skipping corrupted Message
Traceback (most recent call last):
  File "d:\anaconda3\Lib\site-packages\cfgrib\messages.py", line 274, in itervalues
    yield self.filestream.message_from_file(file, errors=errors)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda3\Lib\site-packages\cfgrib\messages.py", line 341, in message_from_file
    return Message.from_file(file, offset, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda3\Lib\site-packages\cfgrib\messages.py", line 97, in from_file
    codes_id = eccodes.codes_grib_new_from_file(file)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda3\Lib\site-packages\gribapi\gribapi.py", line 415, in grib_new_from_file
    GRIB_CHECK(err)
  File "d:\anaconda3\Lib\site-packages\gribapi\gribapi.py", line 232, in GRIB_CHECK
    errors.raise_grib_error(errid)
  File "d:\anaconda3\Lib\site-packages\gribapi\errors.py", line 381, in raise_grib_error
    raise ERROR_M

Processing and coarsening wind_data_2022_12_all_levels.grib...
Processing and coarsening wind_data_2023_01_all_levels.grib...
Processing and coarsening wind_data_2023_01_all_levels.grib...
Processing and coarsening wind_data_2023_02_all_levels.grib...
Processing and coarsening wind_data_2023_02_all_levels.grib...
Processing and coarsening wind_data_2023_03_all_levels.grib...
Processing and coarsening wind_data_2023_03_all_levels.grib...
Processing and coarsening wind_data_2023_04_all_levels.grib...
Processing and coarsening wind_data_2023_04_all_levels.grib...
Processing and coarsening wind_data_2023_05_all_levels.grib...
Processing and coarsening wind_data_2023_05_all_levels.grib...
Processing and coarsening wind_data_2023_06_all_levels.grib...
Processing and coarsening wind_data_2023_06_all_levels.grib...
Processing and coarsening wind_data_2023_07_all_levels.grib...
Processing and coarsening wind_data_2023_07_all_levels.grib...
Processing and coarsening wind_data_2023_08_all_levels.

In [7]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157444752 entries, 0 to 157444751
Data columns (total 12 columns):
 #   Column          Dtype          
---  ------          -----          
 0   time            datetime64[ns] 
 1   isobaricInhPa   float64        
 2   latitude        float64        
 3   longitude       float64        
 4   u               float32        
 5   v               float32        
 6   number          int32          
 7   step            timedelta64[ns]
 8   valid_time      datetime64[ns] 
 9   source_file     object         
 10  wind_speed      float32        
 11  wind_direction  float64        
dtypes: datetime64[ns](2), float32(3), float64(4), int32(1), object(1), timedelta64[ns](1)
memory usage: 11.7+ GB


In [8]:
combined_df.head()

,time,isobaricInhPa,latitude,longitude,u,v,number,step,valid_time,source_file,wind_speed,wind_direction
0,2019-01-01,1000.0,5.875,95.125,-3.999832,-1.854233,0,0 days,2019-01-01,wind_data_2019_01_all_levels.grib,4.408723,245.128618
1,2019-01-01,1000.0,5.875,95.625,-2.395096,0.192642,0,0 days,2019-01-01,wind_data_2019_01_all_levels.grib,2.402831,274.598511
2,2019-01-01,1000.0,5.875,96.125,-2.034012,-0.159164,0,0 days,2019-01-01,wind_data_2019_01_all_levels.grib,2.040230,265.525639
3,2019-01-01,1000.0,5.875,96.625,-2.851151,-0.957504,0,0 days,2019-01-01,wind_data_2019_01_all_levels.grib,3.007636,251.436366
4,2019-01-01,1000.0,5.875,97.125,-1.976639,-0.728989,0,0 days,2019-01-01,wind_data_2019_01_all_levels.grib,2.106781,249.755914


In [ ]:
combined_df['time'].unique()

In [ ]:
# save to csv
output_csv_path = os.path.join(INPUT_FOLDER, f'coarse_wind_data_{START_YEAR}_{END_YEAR}_all_levels.csv')
combined_df.to_csv(output_csv_path, index=False)
print(f"Coarse wind data saved to {output_csv_path}")

# Print summary statistics
print(f"\n--- Data Summary ---")
print(f"Total records: {len(combined_df):,}")
print(f"Pressure levels: {len(combined_df['isobaricInhPa'].unique())} levels")
print(f"Spatial coverage: {len(combined_df[['latitude', 'longitude']].drop_duplicates())} grid points")
print(f"Temporal coverage: {combined_df['time'].nunique()} time steps")
print(f"File size: {os.path.getsize(output_csv_path) / (1024**2):.2f} MB")

# Show wind speed statistics by pressure level
print(f"\n--- Wind Speed Statistics by Pressure Level ---")
wind_stats = combined_df.groupby('isobaricInhPa')['wind_speed'].agg(['mean', 'max', 'std']).round(2)
print(wind_stats.head(10))  # Show first 10 levels

Coarse wind data saved to ../data/wind_data_975hpa/coarse_wind_data.csv
